# Crteate Recommender Schema and Tables

## 1. Database Connection

In [1]:
# Importing libraries
import psycopg2
import pandas as pd

In [2]:
%%capture --no-display capture_output
# Loading SQL
%load_ext sql

In [3]:
%%capture --no-display capture_output
# Loading schemadisplay
%load_ext schemadisplay_magic

In [4]:
%%capture --no-display capture_output
# Configuring SQL so it will load files using Pandas
%config SqlMagic.autopandas=True

In [5]:
%%capture --no-display capture_output
# Set to return floats rather than decimals
DEC2FLOAT = psycopg2.extensions.new_type(
    psycopg2.extensions.DECIMAL.values,
    'DEC2FLOAT',
    lambda value, curs: float(value) if value is not None else None)
psycopg2.extensions.register_type(DEC2FLOAT)

In [6]:
# Defining local variables
DB_ENGINE='postgresql'
DB_HOST='db'
DB_PORT=5432
DB_NAME='recommenderdb'
DB_USER='postgres'
DB_PWD='letmein'

In [7]:
# Creating connection string
DB_ML_SETUP_CONNECTION = '{engine}://{user}:{pwd}@{host}:{port}/{name}'.format(engine=DB_ENGINE,
                                                               user=DB_USER,
                                                               pwd=DB_PWD,
                                                               host=DB_HOST,
                                                               port=DB_PORT,
                                                               name=DB_NAME)

In [8]:
print("Connecting with connection string : {}".format(DB_ML_SETUP_CONNECTION))

%sql $DB_ML_SETUP_CONNECTION

Connecting with connection string : postgresql://postgres:letmein@db:5432/recommenderdb


In [9]:
%%sql

SELECT version();

 * postgresql://postgres:***@db:5432/recommenderdb
1 rows affected.


,version
0,PostgreSQL 13.4 (Debian 13.4-4.pgdg110+1) on x...


In [10]:
%%sql

SHOW search_path;

 * postgresql://postgres:***@db:5432/recommenderdb
1 rows affected.


,search_path
0,"""$user"", public"


In [11]:
%%sql

SELECT *
FROM pg_catalog.pg_tables
WHERE schemaname != 'pg_catalog' AND 
    schemaname != 'information_schema';

 * postgresql://postgres:***@db:5432/recommenderdb
14 rows affected.


,schemaname,tablename,tableowner,tablespace,hasindexes,hasrules,hastriggers,rowsecurity
0,public,django_migrations,postgres,None,True,False,False,False
1,public,django_content_type,postgres,None,True,False,True,False
2,public,auth_permission,postgres,None,True,False,True,False
3,public,auth_group,postgres,None,True,False,True,False
4,public,auth_group_permissions,postgres,None,True,False,True,False
5,recommender,keywords,postgres,None,True,False,True,False
6,public,auth_user_groups,postgres,None,True,False,True,False
7,public,auth_user_user_permissions,postgres,None,True,False,True,False
8,public,django_admin_log,postgres,None,True,False,True,False
9,public,auth_user,postgres,None,True,False,True,False


## 2. Movies Metadata Table Preperation

In [12]:
%%sql

DROP TABLE IF EXISTS movies_metadata CASCADE;

 * postgresql://postgres:***@db:5432/recommenderdb
Done.


""


In [13]:
%%sql

DROP SCHEMA IF EXISTS recommender CASCADE;

 * postgresql://postgres:***@db:5432/recommenderdb
Done.


""


In [14]:
%%sql

CREATE SCHEMA recommender;

 * postgresql://postgres:***@db:5432/recommenderdb
Done.


""


In [15]:
%%sql

CREATE TABLE recommender.movies_metadata(
    adult TEXT,
    belongs_to_collection TEXT,
    budget TEXT,
    genres TEXT,
    homepage TEXT,
    id TEXT,
    imdb_id TEXT,
    original_language TEXT,
    original_title TEXT,
    overview TEXT,
    popularity TEXT,
    poster_path TEXT,
    production_companies TEXT,
    production_countries TEXT,
    release_date TEXT,
    revenue TEXT,
    runtime TEXT,
    spoken_languages TEXT,
    status TEXT,
    tagline TEXT,
    title TEXT,
    video TEXT,
    vote_average TEXT,
    vote_count TEXT
)

 * postgresql://postgres:***@db:5432/recommenderdb
Done.


""


In [16]:
%%capture capture_output

%%sql

SET search_path TO recommender, public;

In [17]:
%%sql

SHOW search_path;

 * postgresql://postgres:***@db:5432/recommenderdb
1 rows affected.


,search_path
0,"recommender, public"


In [18]:
%%sql

\copy movies_metadata(adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count)
FROM './data/clean_movies_metadata.csv'
DELIMITER ','
CSV HEADER;

 * postgresql://postgres:***@db:5432/recommenderdb
0 rows affected.


""


In [19]:
%%sql

SELECT * FROM movies_metadata LIMIT 5;

 * postgresql://postgres:***@db:5432/recommenderdb
5 rows affected.


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,None,Toy Story,False,7.7,5415.0
1,False,None,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",None,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",None,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,None,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",None,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",None,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [20]:
%%sql

DELETE FROM movies_metadata WHERE id = '1997-08-20';
DELETE FROM movies_metadata WHERE id = '2012-09-29';

 * postgresql://postgres:***@db:5432/recommenderdb
1 rows affected.
1 rows affected.


""


In [21]:
%%sql

SELECT id, COUNT(id), title
FROM movies_metadata
GROUP BY id, title
HAVING COUNT(id) > 1
ORDER BY COUNT(id);

 * postgresql://postgres:***@db:5432/recommenderdb
22 rows affected.


,id,count,title
0,84198,2,A Place at the Table
1,15028,2,Clockstoppers
2,23305,2,The Warrior
3,69234,2,The Phantom of the Opera
4,11115,2,Deal
5,168538,2,"Nana, the True Key of Pleasure"
6,14788,2,Bubble
7,152795,2,The Congress
8,18440,2,Days of Darkness
9,132641,2,Wife


In [22]:
%%sql

DELETE FROM movies_metadata
WHERE id IN
    (SELECT id
    FROM 
        (SELECT id,
         ROW_NUMBER() OVER( PARTITION BY id
        ORDER BY id ) AS row_num
        FROM movies_metadata ) t
        WHERE t.row_num > 1 );

 * postgresql://postgres:***@db:5432/recommenderdb
45 rows affected.


""


In [23]:
%%sql

ALTER TABLE movies_metadata ALTER COLUMN id TYPE NUMERIC USING id::numeric;

 * postgresql://postgres:***@db:5432/recommenderdb
Done.


""


In [24]:
%%sql

ALTER TABLE movies_metadata ADD CONSTRAINT movies_metadata_pk PRIMARY KEY (id);

 * postgresql://postgres:***@db:5432/recommenderdb
Done.


""


In [25]:
%%sql

ALTER TABLE movies_metadata ALTER COLUMN adult TYPE BOOLEAN USING adult::boolean;

 * postgresql://postgres:***@db:5432/recommenderdb
Done.


""


In [26]:
%%sql

ALTER TABLE movies_metadata ALTER COLUMN budget TYPE NUMERIC USING budget::numeric;

 * postgresql://postgres:***@db:5432/recommenderdb
Done.


""


In [27]:
%%sql

ALTER TABLE movies_metadata ALTER COLUMN original_title TYPE VARCHAR(255);

 * postgresql://postgres:***@db:5432/recommenderdb
Done.


""


In [28]:
%%sql

ALTER TABLE movies_metadata ALTER COLUMN popularity TYPE DECIMAL USING popularity::decimal;

 * postgresql://postgres:***@db:5432/recommenderdb
Done.


""


In [29]:
%%sql

ALTER TABLE movies_metadata ALTER COLUMN release_date TYPE DATE USING release_date::date;

 * postgresql://postgres:***@db:5432/recommenderdb
Done.


""


In [30]:
%%sql

ALTER TABLE movies_metadata ALTER COLUMN revenue TYPE MONEY USING revenue::money;

 * postgresql://postgres:***@db:5432/recommenderdb
Done.


""


In [31]:
%%sql

ALTER TABLE movies_metadata ALTER COLUMN runtime TYPE DECIMAL USING runtime::decimal;

 * postgresql://postgres:***@db:5432/recommenderdb
Done.


""


In [32]:
%%sql

ALTER TABLE movies_metadata ALTER COLUMN title TYPE VARCHAR(255);

 * postgresql://postgres:***@db:5432/recommenderdb
Done.


""


In [33]:
%%sql

ALTER TABLE movies_metadata ALTER COLUMN vote_average TYPE DECIMAL USING vote_average::decimal;

 * postgresql://postgres:***@db:5432/recommenderdb
Done.


""


In [34]:
%%sql

ALTER TABLE movies_metadata ALTER COLUMN vote_count TYPE DECIMAL USING vote_count::decimal;

 * postgresql://postgres:***@db:5432/recommenderdb
Done.


""


## 3. Credits Table Preperation

In [35]:
%%sql

DROP TABLE IF EXISTS recommender.credits CASCADE;

 * postgresql://postgres:***@db:5432/recommenderdb
Done.


""


In [36]:
%%sql

CREATE TABLE recommender.credits(
    "cast" TEXT,
    crew TEXT,
    id TEXT
)

 * postgresql://postgres:***@db:5432/recommenderdb
Done.


""


In [37]:
%%sql

\copy recommender.credits("cast",crew,id)
FROM './data/clean_credits.csv'
DELIMITER ','
CSV HEADER;

 * postgresql://postgres:***@db:5432/recommenderdb
0 rows affected.


""


In [38]:
%%sql

SELECT id, COUNT(id)
FROM recommender.credits
GROUP BY id
HAVING COUNT(id) > 1
ORDER BY COUNT(id);

 * postgresql://postgres:***@db:5432/recommenderdb
43 rows affected.


,id,count
0,99080,2
1,10991,2
2,109962,2
3,110428,2
4,11115,2
5,116723,2
6,11752,2
7,119916,2
8,123634,2
9,125458,2


In [39]:
%%sql

DELETE FROM recommender.credits
WHERE id IN
    (SELECT id
    FROM 
        (SELECT id,
         ROW_NUMBER() OVER( PARTITION BY id
        ORDER BY id ) AS row_num
        FROM recommender.credits ) t
        WHERE t.row_num > 1 );

 * postgresql://postgres:***@db:5432/recommenderdb
87 rows affected.


""


In [40]:
%%sql

ALTER TABLE recommender.credits ADD CONSTRAINT credits_pk PRIMARY KEY (id);

 * postgresql://postgres:***@db:5432/recommenderdb
Done.


""


In [41]:
%%sql

ALTER TABLE recommender.credits ALTER COLUMN id TYPE NUMERIC USING id::numeric;

 * postgresql://postgres:***@db:5432/recommenderdb
Done.


""


In [42]:
%%sql

DELETE FROM recommender.credits 
    WHERE id NOT IN (
        SELECT ID FROM recommender.movies_metadata
    )

 * postgresql://postgres:***@db:5432/recommenderdb
10457 rows affected.


""


In [43]:
%%sql

ALTER TABLE recommender.credits
ADD CONSTRAINT id_fk FOREIGN KEY (id) REFERENCES recommender.movies_metadata(id);

 * postgresql://postgres:***@db:5432/recommenderdb
Done.


""


## 4. Keywords Table Preperation

In [44]:
%%sql

DROP TABLE IF EXISTS recommender.keywords CASCADE;

 * postgresql://postgres:***@db:5432/recommenderdb
Done.


""


In [45]:
%%sql

CREATE TABLE recommender.keywords(
    id TEXT,
    keywords TEXT
)

 * postgresql://postgres:***@db:5432/recommenderdb
Done.


""


In [46]:
%%sql

\copy recommender.keywords(id,keywords)
FROM './data/clean_keywords.csv'
DELIMITER ','
CSV HEADER;

 * postgresql://postgres:***@db:5432/recommenderdb
0 rows affected.


""


In [47]:
%%sql

SELECT id, COUNT(id)
FROM recommender.keywords
GROUP BY id
HAVING COUNT(id) > 1
ORDER BY COUNT(id);

 * postgresql://postgres:***@db:5432/recommenderdb
985 rows affected.


,id,count
0,42588,2
1,84198,2
2,155336,2
3,10991,2
4,42495,2
...,...,...
980,181426,2
981,61464,2
982,83001,2
983,159849,3


In [48]:
%%sql

DELETE FROM recommender.keywords
WHERE id IN
    (SELECT id
    FROM 
        (SELECT id,
         ROW_NUMBER() OVER( PARTITION BY id
        ORDER BY id ) AS row_num
        FROM recommender.keywords ) t
        WHERE t.row_num > 1 );

 * postgresql://postgres:***@db:5432/recommenderdb
1972 rows affected.


""


In [49]:
%%sql

ALTER TABLE recommender.keywords ADD CONSTRAINT keywords_pk PRIMARY KEY (id);

 * postgresql://postgres:***@db:5432/recommenderdb
Done.


""


In [50]:
%%sql

ALTER TABLE recommender.keywords ALTER COLUMN id TYPE NUMERIC USING id::numeric;

 * postgresql://postgres:***@db:5432/recommenderdb
Done.


""


In [51]:
%%sql

DELETE FROM recommender.keywords 
    WHERE id NOT IN (
        SELECT ID FROM recommender.movies_metadata
    )

 * postgresql://postgres:***@db:5432/recommenderdb
9501 rows affected.


""


In [52]:
%%sql

ALTER TABLE recommender.keywords
ADD CONSTRAINT id_fk FOREIGN KEY (id) REFERENCES recommender.movies_metadata(id);

 * postgresql://postgres:***@db:5432/recommenderdb
Done.


""
